# Stars: **

### Day 16: The Floor Will Be Lava

#### Part 1

In [2]:
def get_data(file_name):
    with open(file_name, "r") as file:
        text = file.read() 

    lines = text.split("\n")
    grid = [list(line) for line in lines]
    return grid 

In [3]:
def display_grid(grid):
    for row in grid:
        print("".join(row))

In [4]:
dir_offsets = {
    "Right": [0,1],
    "Left":[0,-1],
    "Up":[-1,0],
    "Down":[1,0]
}

In [5]:
def contains_negative(lst):
    for item in lst:
        if item < 0:
            return True
    return False

In [6]:
def move_pos(grid, pos, direction):
    
    offset = dir_offsets[direction]
    
    row_index = pos[0]
    col_index = pos[1]
    row_index_offset = row_index + offset[0]
    col_index_offset = col_index + offset[1]
    
    next_pos = [row_index_offset, col_index_offset]
    next_val = grid[row_index_offset][col_index_offset]

    return next_pos, next_val

In [7]:
def get_next_dir(val, dir):

    if val == ".":
        val_type = "Empty Space"
        next_dir = dir

    elif val == "|":
        val_type = "Vertical Splitter"
        if dir == "Right" or dir == "Left":
            next_dir = ["Up", "Down"]
        else:
            next_dir = dir
    
    elif val == "-":
        val_type = "Horizontal Splitter"
        if dir == "Up" or dir == "Down":
            next_dir = ["Right", "Left"]
        else:
            next_dir = dir

    elif val == "/":
        val_type = "Back Mirror"
        if dir == "Right":
            next_dir = "Up"
        elif dir == "Left":
            next_dir = "Down"
        elif dir == "Up":
            next_dir = "Right"
        elif dir == "Down":
            next_dir = "Left"

    elif val == "\\":
        val_type = "Forward Mirror"
        if dir == "Right":
            next_dir = "Down"
        elif dir == "Left":
            next_dir = "Up"
        elif dir == "Up":
            next_dir = "Left"
        elif dir == "Down":
            next_dir = "Right"
    
    else:
        val_type = "Unknown"
        next_dir = "Unknown"

    return val_type, next_dir

In [18]:
file_name = "input.txt"
grid = get_data(file_name)
hash_grid = get_data(file_name)

start_pos = [0,0]
pos_list = {0:[start_pos]}

start_dir = "Right"
dir_list = {0:[start_dir]}

visited_splitters = []

print("Start\n")
display_grid(grid)
print("")

key = 0
iter = 0
while True:

    try: 
        sub_hash_grid = get_data(file_name)
        lst = pos_list[key]
        # print(lst)

        for i, pos in enumerate(lst):
            if i == (len(lst) - 1):

                # print("Next pos:", pos)

                # get direction moved from
                dir = dir_list[key][-1]
                
                # get current coordinates
                row_index = pos[0]
                col_index = pos[1]

                # replace current position with hash (#)
                hash_grid[row_index][col_index] = "#"
                sub_hash_grid[row_index][col_index] = "#"
                # print("")
                # display_grid(hash_grid)
                # print("")
                # display_grid(sub_hash_grid)
                # print("")

                # get current value
                val = grid[row_index][col_index]
                
                # get value type and next direction
                val_type, next_dirs = get_next_dir(val, dir)

                # check if a splitter and whether it's been visited before
                if ((dir=="Right" or dir=="Left") and val=="|") or ((dir=="Up" or dir=="Down") and val=="-"):
                    # print(f"Splitter {pos}")
                    if pos in visited_splitters:
                        # print(" >>> End of path: Splitter already visited")
                        iter += 1
                        continue
                    else:
                        visited_splitters.append(pos)

                # printing
                # print("---------------------------------------------------------------------")
                # print(f"\nPath {key} Step {i+1}: moved '{dir}' into {pos} of value '{val}' ({val_type})")
                # print()

                # check if split into two paths
                if isinstance(next_dirs, list):

                    max_key = max(dir_list.keys())
                    # print("List", next_dirs, "Max key", max_key)

                    valid_next_dir = []

                    for dir_index, dir_item in enumerate(next_dirs):

                        try:
                            next_pos, next_val = move_pos(grid, pos, direction=dir_item)

                        except IndexError:
                            # print(" >>> End of path: Out of index")
                            pass
                            iter += 1
                            continue

                        if contains_negative(next_pos):
                            # print(f"{dir_item} CONTAINS NEGATIVE")
                            pass
                        else:
                            valid_next_dir.append([dir_item, next_pos])
                    
                    if len(valid_next_dir) == 1:
                        # print(valid_next_dir)
                        dir_list[key].append(valid_next_dir[0][0])
                        pos_list[key].append(valid_next_dir[0][1])
        
                    elif len(valid_next_dir) == 2:
                        for dir_index, dir_item in reversed(list(enumerate(next_dirs))): 
                            if dir_index == 0:
                                dir_list[key].append(valid_next_dir[0][0])
                                pos_list[key].append(valid_next_dir[0][1])
                            elif dir_index == 1:
                                new_dir_item = dir_list[key] + [dir_item]
                                new_pos_item = pos_list[key] + [next_pos]
                                dir_list[max_key+1] = new_dir_item
                                pos_list[max_key+1] = new_pos_item
                    
                    else: 
                        # print("No valid directions")
                        pass
                        
                else:
                    next_dir = next_dirs
                    dir_list[key].append(next_dir) 

                    try:
                        next_pos, next_val = move_pos(grid, pos, direction=next_dir)

                    except IndexError:
                        # print(" >>> End of path: Out of index")
                        iter += 1
                        continue

                    if contains_negative(next_pos):
                        # print("CONTAINS NEGATIVE")
                        pass
                    else:
                        pos_list[key].append(next_pos)
                        
                # # print paths
                # print("Directions list:")
                # for path_dir_key, path_dir_val in dir_list.items():
                #     print("", path_dir_key, path_dir_val)
                # print("Positions list:")
                # for path_pos_key, path_pos_val in pos_list.items():
                #     print("", path_pos_key, path_pos_val)

                # # print current path
                # print("Directions:", dir_list[key])
                # print("Positions:", pos_list[key])
                # print("Splitters visited:", visited_splitters)

                # # replace current position with hash (#)
                # hash_grid[row_index][col_index] = "#"

                # # printing
                # print("")
                # display_grid(hash_grid)
                # print("")
                # display_grid(sub_hash_grid)

    except KeyError:
        break

    key += 1

    if key == 20000:
        break

print("Final Path\n")
display_grid(hash_grid)

hash_count = 0
for i, row in enumerate(hash_grid):
    for j, col in enumerate(hash_grid):
        if hash_grid[i][j] == "#":
            hash_count += 1
print(f"\nHash count = {hash_count}")

Start

\..|.....................|.-........\.............-................|./....|.....-...-.........................
.......\.........|..............-...||..............-...........--.|...............||..........\............|.
........../.....................\.........\..................\....|.....-.................../..-.\............
................/..../........../..-....|\.|...-...................\....|/........|...........................
.....|../-\.|....|................................--.-./..........|.........../..-....-................-....|.
.../..........\................-.......\................|/..\.................\.....\...|....../............./
..-../.../......-.....\........//.............|.........../...|.........../.......-...........................
......\....../......./..|\............/..........................\..............\.....\.........|......-...|..
.|/.......\.........|...../..//...................................-....-...........././-...............-.

### Part 2

In [42]:
def get_hash_count(file_name="input.txt", start_pos=[0,0], start_dir="Right", print_grid=False):

    grid = get_data(file_name)
    hash_grid = get_data(file_name)

    pos_list = {0:[start_pos]}
    dir_list = {0:[start_dir]}

    visited_splitters = []

    if print_grid == True:
        print("Start\n")
        display_grid(grid)
        print("")

    key = 0
    iter = 0
    while True:

        try: 
            sub_hash_grid = get_data(file_name)
            lst = pos_list[key]

            for i, pos in enumerate(lst):
                if i == (len(lst) - 1):

                    # get direction moved from
                    dir = dir_list[key][-1]
                    
                    # get current coordinates
                    row_index = pos[0]
                    col_index = pos[1]

                    # replace current position with hash (#)
                    hash_grid[row_index][col_index] = "#"
                    sub_hash_grid[row_index][col_index] = "#"

                    # get current value
                    val = grid[row_index][col_index]
                    
                    # get value type and next direction
                    val_type, next_dirs = get_next_dir(val, dir)

                    # check if a splitter and whether it's been visited before
                    if ((dir=="Right" or dir=="Left") and val=="|") or ((dir=="Up" or dir=="Down") and val=="-"):
                        if pos in visited_splitters:
                            iter += 1
                            continue
                        else:
                            visited_splitters.append(pos)

                    # check if split into two paths
                    if isinstance(next_dirs, list):

                        max_key = max(dir_list.keys())

                        valid_next_dir = []

                        for dir_index, dir_item in enumerate(next_dirs):

                            try:
                                next_pos, next_val = move_pos(grid, pos, direction=dir_item)

                            except IndexError:
                                pass
                                iter += 1
                                continue

                            if contains_negative(next_pos):
                                pass
                            else:
                                valid_next_dir.append([dir_item, next_pos])
                        
                        if len(valid_next_dir) == 1:
                            dir_list[key].append(valid_next_dir[0][0])
                            pos_list[key].append(valid_next_dir[0][1])
            
                        elif len(valid_next_dir) == 2:
                            for dir_index, dir_item in reversed(list(enumerate(next_dirs))): 
                                if dir_index == 0:
                                    dir_list[key].append(valid_next_dir[0][0])
                                    pos_list[key].append(valid_next_dir[0][1])
                                elif dir_index == 1:
                                    new_dir_item = dir_list[key] + [dir_item]
                                    new_pos_item = pos_list[key] + [next_pos]
                                    dir_list[max_key+1] = new_dir_item
                                    pos_list[max_key+1] = new_pos_item
                        
                        else: 
                            pass
                            
                    else:
                        next_dir = next_dirs
                        dir_list[key].append(next_dir) 

                        try:
                            next_pos, next_val = move_pos(grid, pos, direction=next_dir)

                        except IndexError:
                            iter += 1
                            continue

                        if contains_negative(next_pos):
                            pass
                        else:
                            pos_list[key].append(next_pos)

        except KeyError:
            break

        key += 1

        if key == 20000:
            break

    hash_count = 0
    for i, row in enumerate(hash_grid):
        for j, col in enumerate(hash_grid):
            if hash_grid[i][j] == "#":
                hash_count += 1

    if print_grid == True:
        print("Final Path\n")
        display_grid(hash_grid)
        print(f"\nHash count = {hash_count}")
        
    return hash_count



In [43]:
get_hash_count(file_name="test_input.txt", start_pos=[0,0], start_dir="Right", print_grid=False)

46

In [44]:
corners = [
    [0,0],
    [0,-1],
    [-1,0],
    [-1,-1]
]

In [45]:
sides = {
    "Top":"Down", 
    "Bottom": "Up", 
    "Leftmost": "Right", 
    "Rightmost": "Left"
}

In [ ]:
for col_index in range(len(grid[0])):

In [69]:
file_name = "input.txt"
grid = get_data(file_name)
display_grid(grid)

\..|.....................|.-........\.............-................|./....|.....-...-.........................
.......\.........|..............-...||..............-...........--.|...............||..........\............|.
........../.....................\.........\..................\....|.....-.................../..-.\............
................/..../........../..-....|\.|...-...................\....|/........|...........................
.....|../-\.|....|................................--.-./..........|.........../..-....-................-....|.
.../..........\................-.......\................|/..\.................\.....\...|....../............./
..-../.../......-.....\........//.............|.........../...|.........../.......-...........................
......\....../......./..|\............/..........................\..............\.....\.........|......-...|..
.|/.......\.........|...../..//...................................-....-...........././-...............-......
.

In [70]:
max_hash_count = 0
max_config = {0:[]}
for key in sides.keys():
    dir = sides[key]
    if key == "Top" or key == "Bottom":
        for col_index in range(len(grid[0])):
            if key == "Top": 
                row_index = 0
            else:
                row_index = len(grid)-1
            hash_count = get_hash_count(file_name=file_name, start_pos=[row_index, col_index], start_dir=dir, print_grid=False)
            if hash_count > max_hash_count:
                max_hash_count = hash_count
                max_config[0] = [key, dir, hash_count, [row_index, col_index]]
    elif key == "Leftmost" or key == "Rightmost":
        for row_index in range(len(grid)):
            if key == "Leftmost": 
                col_index = 0
            else:
                col_index = len(grid[0])-1
            hash_count = get_hash_count(file_name=file_name, start_pos=[row_index, col_index], start_dir=dir, print_grid=False)
            if hash_count > max_hash_count:
                max_hash_count = hash_count
                max_config[0] = [key, dir, hash_count, [row_index, col_index]]

print("\nMax hash count =", max_hash_count, max_config)


Max hash count = 8754 {0: ['Rightmost', 'Left', 8754, [93, 109]]}
